In [25]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import beta


In [92]:
df = pd.read_csv('torvik_2025-12-11_16-43-07.csv')

In [93]:
#Use standard scaler on sos
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df['sos'] = scaler.fit_transform(df['sos'].values.reshape(-1,1))

In [123]:
ftr = pd.read_csv('Raw Data/FT 2026.csv')
ftr

#Replace state with St. in ftr if not N.C. State

ftr['School'] = ftr['School'].str.replace('State','St.')

#If name N.C. St. replace with N.C. State
ftr['School'] = ftr['School'].str.replace('N.C. St.','N.C. State')

#Trim spaces off school names at end
ftr['School'] = ftr['School'].str.strip()

In [95]:
def get_adj_metric_pos (df, metric):
    X = sm.add_constant(df['sos'])
    model = sm.OLS(df[metric], X).fit()
    df['Adj ' + metric] = df[metric] + (model.params['sos'] * df['sos'])
    return df
def get_adj_metric_neg (df, metric):
    X = sm.add_constant(df['sos'])
    model = sm.OLS(df[metric], X).fit()
    df['Adj ' + metric] = df[metric] - (model.params['sos'] * df['sos'])
    return df
def harmonic_mean(x,y):
    return ((x*y*2)/(x+y))

In [96]:
df.columns

Index(['Team', 'eFG%', 'eFG% Def', 'FTR', 'FTR Def', 'OR%', 'DR%', 'TO%',
       'TO% Def.', '3P%', '3pD%', '2p%', '2p%D', 'ft%', 'ft%D', '3P rate',
       '3P rate D', 'arate', 'arateD', 'rank', 'team', 'conf', 'record',
       'adjoe', 'adjde', 'barthag', 'sos', 'adjt', 'Year', 'EFG Z', 'EFGD Z',
       'TOR Z', 'TORD Z', 'ORB Z', 'DRB Z', 'FTR Z', 'FTRD Z', '2P Z', '2PD Z',
       '3P Z', '3PD Z', '3PR Z', '3PRD Z', 'Adj T Z'],
      dtype='object')

In [97]:
#Find coefficients to adjust metrics based on SOS
df = get_adj_metric_pos(df, 'OR%')
df = get_adj_metric_pos(df, 'eFG%')
df = get_adj_metric_pos(df, '2p%')
df = get_adj_metric_pos(df, '3P%')
df = get_adj_metric_pos(df, 'TO% Def.')
df = get_adj_metric_pos(df, 'DR%')
df = get_adj_metric_pos(df, 'eFG% Def')
df = get_adj_metric_pos(df, '2p%D')
df = get_adj_metric_pos(df, '3pD%')
df = get_adj_metric_pos(df, 'TO%')

In [98]:
def get_beta_value (df,metric,home_team,away_team):
    mean_home = df.loc[home_team][metric]
    mean_away = df.loc[away_team][metric]
    alpha_home = 20 * mean_home
    beta_home = 20 * (1 - mean_home)
    alpha_away = 20 * mean_away
    beta_away = 20 * (1 - mean_away)
    home_val = beta.rvs(alpha_home, beta_home, size=1)
    away_val = beta.rvs(alpha_away, beta_away, size=1)
    return home_val[0], away_val[0]

def get_regular_team_stats(home_team,away_team):
    df_team = df[(df['Team'] == home_team) | (df['Team'] == away_team)]
    df_team = df_team[['Team', 'barthag', 'Adj OR%', 'Adj DR%', 'Adj eFG%', 'Adj 2p%', 'Adj 3P%', 'Adj TO% Def.', 'Adj eFG% Def', 'Adj TO%', 'Adj 2p%D', 'Adj 3pD%','adjt','3P rate','FTR','ft%D','3P rate D']]
    df_team = df_team.set_index('Team')
    for col in [ 'Adj OR%', 'Adj DR%', 'Adj eFG%', 'Adj 2p%', 'Adj 3P%', 'Adj TO% Def.', 'Adj eFG% Def', 'Adj TO%', 'Adj 2p%D', 'Adj 3pD%','3P rate','FTR','ft%D','3P rate D']:
        df_team[col] = df_team[col] / 100
    return df_team

def get_simulated_team_stats(home_team, away_team, df):
    df_team = df[(df['Team'] == home_team) | (df['Team'] == away_team)]
    df_team = df_team[['Team', 'barthag' ,'Adj OR%', 'Adj DR%', 'Adj eFG%', 'Adj 2p%', 'Adj 3P%', 'Adj TO% Def.', 'Adj eFG% Def', 'Adj TO%', 'Adj 2p%D', 'Adj 3pD%','adjt','3P rate','FTR','ft%D','3P rate D']]
    df_team = df_team.set_index('Team')
    #Reorder rows such that home team is first
    if df_team.index[0] == away_team:
        df_team = df_team.reindex([home_team, away_team])
    df_sim = df_team.copy()
    for col in [ 'Adj OR%', 'Adj DR%', 'Adj eFG%', 'Adj 2p%', 'Adj 3P%', 'Adj TO% Def.', 'Adj eFG% Def', 'Adj TO%', 'Adj 2p%D', 'Adj 3pD%','3P rate','FTR','ft%D','3P rate D']:
        df_team[col] = df_team[col] / 100
        df_sim[col] = df_sim[col] / 100
        df_sim[col] = get_beta_value(df_sim,col,home_team,away_team)
    return df_sim

In [99]:
def game_spread_ppp (home_team, away_team, teams_df, use_bart = True):

    

    #For each team take the harmonic mean of 
    df_game = pd.DataFrame()
    df_game['Team'] = [home_team, away_team]

    
    #Get free throw rates for home and away teams
    home_ftr = ftr[ftr['School'] == home_team]['FT%'].values[0]
    away_ftr = ftr[ftr['School'] == away_team]['FT%'].values[0]

    #Get strength differences
    h_w = 1
    home_advantage = (teams_df.loc[home_team]['barthag'] - teams_df.loc[away_team]['barthag'])/2.5 + 1
    if use_bart:
        home_advantage = home_advantage
    else:
        home_advantage = 1
    a_w = 1#(teams_df.loc[away_team]['barthag'] - teams_df.loc[home_team]['barthag']) + 1
    
    #Get harmonic means for each team
    df_game['ORB'] = [harmonic_mean(teams_df.loc[home_team]['Adj OR%'],teams_df.loc[away_team]['Adj DR%']),
                    harmonic_mean(teams_df.loc[away_team]['Adj OR%'],teams_df.loc[home_team]['Adj DR%'])]
    df_game['2P%'] = [harmonic_mean(teams_df.loc[home_team]['Adj 2p%'],teams_df.loc[away_team]['Adj 2p%D']),
                    harmonic_mean(teams_df.loc[away_team]['Adj 2p%'],teams_df.loc[home_team]['Adj 2p%D'])]
    df_game['3P%'] = [harmonic_mean(teams_df.loc[home_team]['Adj 3P%'],teams_df.loc[away_team]['Adj 3pD%']),
                    harmonic_mean(teams_df.loc[away_team]['Adj 3P%'],teams_df.loc[home_team]['Adj 3pD%'])]
    df_game['TOR'] = [harmonic_mean(teams_df.loc[home_team]['Adj TO%'],teams_df.loc[away_team]['Adj TO% Def.']),
                    harmonic_mean(teams_df.loc[away_team]['Adj TO%'],teams_df.loc[home_team]['Adj TO% Def.'])]
    df_game['FTR'] = [harmonic_mean(teams_df.loc[home_team]['FTR'],teams_df.loc[away_team]['ft%D']),
                    harmonic_mean(teams_df.loc[away_team]['FTR'],teams_df.loc[home_team]['ft%D'])]
    df_game['Adj FTR'] = df_game['FTR'] * .44
    df_game['FT%'] = [home_ftr, away_ftr]
    df_game['3PR'] = [harmonic_mean(teams_df.loc[home_team]['3P rate'],teams_df.loc[away_team]['3P rate D']),
                    harmonic_mean(teams_df.loc[away_team]['3P rate'],teams_df.loc[home_team]['3P rate D'])]
    df_game['ev_2pt'] = df_game['2P%'] * (1 - df_game['3PR']) * 2
    df_game['ev_3pt'] = df_game['3P%'] * df_game['3PR'] * 3
    df_game['Adj ORB'] = ((df_game['2P%'] * (1 - df_game['3PR']) + (df_game['3P%'] * df_game['3PR']))) * df_game['ORB']    
    df_game['shot attempts'] = (((1-df_game['TOR'] - df_game['Adj FTR'] - df_game['Adj ORB']) * 1)) + (df_game['Adj ORB']*2)
    df_game['adjt'] = [teams_df.loc[home_team]['adjt'], teams_df.loc[away_team]['adjt']]
    df_game['PPP'] = df_game['shot attempts'] * df_game['ev_2pt'] + df_game['shot attempts'] * df_game['ev_3pt'] + (df_game['Adj FTR'] * df_game['FT%'] * 1.5)
    df_game['Est FTA'] = (df_game['Adj FTR'] * 1.5 * df_game['adjt']) 

    possessions = (teams_df.loc[home_team]['adjt']+teams_df.loc[away_team]['adjt'])/2

    df_game['Predicted Score'] = df_game['PPP'] * possessions

    #Update home team to multiply by home_advantage if use_bart = True

    df_game.loc[0,'Predicted Score'] = df_game.loc[0,'Predicted Score'] * home_advantage

    #Get projected score for home team

    #print('Projected Spread Home Team: ', df_game.loc[1]['Predicted Score'] - (df_game.loc[0]['Predicted Score'] + 3))
    #print('Projected Total: ', df_game.loc[0]['Predicted Score'] + df_game.loc[1]['Predicted Score'])

    return df_game

In [100]:
def probability_to_moneyline(win_prob):
    """
    Convert win probability (0 to 1) to American moneyline odds.
    
    Args:
        win_prob (float): The probability of winning (between 0 and 1).
        
    Returns:
        int: The moneyline odds.
    """
    if not 0 < win_prob < 1:
        raise ValueError("Win probability must be between 0 and 1 (exclusive).")
        
    if win_prob > 0.5:
        # Negative odds for favorites
        moneyline = -100 * (win_prob / (1 - win_prob))
    else:
        # Positive odds for underdogs
        moneyline = 100 * ((1 - win_prob) / win_prob)
        
    return int(round(moneyline))

def get_betting_pcts (spread_home, vegas_ou,spread,totals):

    
    spread = np.array(spread)
    totals = np.array(totals)

    if spread_home > 0:
        print('Odds of Home Covering spread:', len(spread[spread <= spread_home])/len(spread))
    else:
        print('Odds of Home Covering spread:', len(spread[spread <= spread_home])/len(spread))
    if spread_home > 0:
        print('Odds of Away Covering spread:', len(spread[spread >= spread_home])/len(spread))
    else:
        print('Odds of Away Covering spread:', len(spread[spread >= spread_home])/len(spread))

    print('Odds of Over: ', len(totals[totals >= vegas_ou])/len(totals))
    print('Odds of Under: ', len(totals[totals <= vegas_ou])/len(totals))

import scipy.stats as stats
def calculate_win_probability(proj_spread, vegas_spread, sigma=4.5):
    """Calculate win probability based on projected and Vegas spread."""
    diff = vegas_spread - proj_spread  # Difference between Vegas and projected spread
    return stats.norm.cdf(diff / sigma)  # Normal CDF for probability

def calculate_ev(win_prob, odds=-110):
    """Calculate Expected Value (EV) given win probability and betting odds."""
    payout = 100 if odds == -110 else abs(10000 / odds)  # Convert odds to payout per $100 risked
    risked = 110  # Standard -110 odds require risking 110 to win 100
    return (win_prob * payout) - ((1 - win_prob) * risked)

In [101]:
def simulate_game(home_team, away_team,num_sims = 1000,home_adjustment = 3,away_adjustment = 0, use_bart = True):
    home_wins = 0
    away_wins = 0
    spread = []
    totals = []
    ties = 0
    for i in range(num_sims):
        df_team = get_simulated_team_stats(home_team, away_team, df)
        df_game = game_spread_ppp(home_team, away_team, df_team, use_bart)
        #home_advantage = (df_team.loc[home_team]['Barthag'] - df_team.loc[away_team]['Barthag'])/2
        if (df_game.loc[0]['Predicted Score'] + home_adjustment)  > df_game.loc[1]['Predicted Score'] + away_adjustment:
            home_wins += 1
        elif (df_game.loc[0]['Predicted Score'] + home_adjustment) < df_game.loc[1]['Predicted Score'] + away_adjustment:
            away_wins += 1
        else:
            ties += 1
        spread.append(df_game.loc[1]['Predicted Score'] + away_adjustment - ((df_game.loc[0]['Predicted Score'] + home_adjustment) ))
        totals.append((df_game.loc[0]['Predicted Score'] + home_adjustment)  + df_game.loc[1]['Predicted Score'])
    
    print(home_team + ' Wins: ', home_wins/num_sims)
    print(away_team + ' Wins: ', away_wins/num_sims)
    print('Ties: ', ties/num_sims)
    print('Median Spread: ', np.median(spread))
    print('Median Total: ', np.median(totals))
    print('Average Spread: ', np.mean(spread))
    print('Average Total: ', np.mean(totals))
    print('Money Line Home Team: ',probability_to_moneyline(home_wins/num_sims))


    #plt.hist(spread, bins=50)
    #plt.show()
    #plt.hist(totals, bins=50)
    #plt.show()

    return spread,totals

In [102]:
def get_efficiency_pred(home_team, away_team):
    df_game = df[(df['Team'] == home_team) | (df['Team'] == away_team)]
    df_game = df_game[['Team','adjoe','adjde','adjt']]
    df_game = df_game.set_index('Team')
    #Make sure home team is first
    if df_game.index[0] == away_team:
        df_game = df_game.reindex([home_team, away_team])
    expected_possessions = df_game['adjt'].sum() / 2

    df_game['Expected PP100'] = [harmonic_mean(df_game.loc[home_team]['adjoe'],df_game.loc[away_team]['adjde']),
                        harmonic_mean(df_game.loc[away_team]['adjoe'],df_game.loc[home_team]['adjde'])]
    df_game['Score'] = (df_game['Expected PP100'] * expected_possessions)/100
    df_game['Score']
    spread = df_game.loc[away_team]['Score'] - (df_game.loc[home_team]['Score'] + 3)
    total = df_game['Score'].sum()
    return spread,total

In [127]:
home_team = "Utah"
away_team = "Mississippi St."

In [232]:
df_mean = get_regular_team_stats(home_team, away_team)
game_spread_ppp(home_team, away_team,df_mean,use_bart=True)

,Team,ORB,2P%,3P%,TOR,FTR,Adj FTR,FT%,3PR,ev_2pt,ev_3pt,Adj ORB,shot attempts,adjt,PPP,Est FTA,Predicted Score
0,Kentucky,0.310983,0.477815,0.341481,0.148551,0.386300,0.169972,0.755,0.385821,0.586928,0.395252,0.132234,0.813712,71.209967,0.991704,18.155541,71.124831
1,North Carolina,0.266082,0.489582,0.295018,0.155202,0.534581,0.235216,0.710,0.442088,0.546287,0.391272,0.107382,0.716964,70.971622,0.922701,25.040453,65.595558


In [226]:
get_efficiency_pred(home_team,away_team)

(-1.9673439087381865, 142.40533005669766)

In [128]:
spread,totals = simulate_game(home_team,away_team,num_sims = 1000,home_adjustment=3 , use_bart=True)

Utah Wins:  0.445
Mississippi St. Wins:  0.555
Ties:  0.0
Median Spread:  1.6990083711221544
Median Total:  137.38378136320912
Average Spread:  1.5562812387968419
Average Total:  137.5194895103176
Money Line Home Team:  125


In [107]:
np.mean(spread)

-9.522943050125832

In [82]:
((5 + 7.5)/2)*.75 + 6.5*.25

6.3125

In [83]:
wp = calculate_win_probability(6.3125,6.5)
print(wp)
ev = calculate_ev(wp, odds=-110)
print(ev)

0.5166177864903634
-1.5102648370236906


In [18]:
get_betting_pcts(-14.5,157.5,spread,totals)

Odds of Home Covering spread: 0.789
Odds of Away Covering spread: 0.211
Odds of Over:  0.504
Odds of Under:  0.496


In [266]:
#Find rows where opponent is not none
matchups = pd.read_csv('12-03-2025 Matchups.csv')

In [267]:
matchups

,Home,Away,Home_Advantage
0,Minnesota,Indiana,2.0
1,Arkansas,Louisville,3.0
2,Alabama,Clemson,3.0
3,Vanderbilt,SMU,3.0
4,Auburn,N.C. St.,3.0
5,Akron,Bucknell,2.0
6,Wisconsin,Northwestern,3.0
7,Texas,Virginia,3.0
8,Georgia Tech,Mississippi St.,2.5
9,Washington,UCLA,2.0


In [268]:
#Search home and away teams in df, if missing delete row
for index, row in matchups.iterrows():
    home_team = row['Home']
    away_team = row['Away']
    if home_team not in df['Team'].values or away_team not in df['Team'].values:
        matchups = matchups.drop(index)
    #Check ftr as well
    if home_team not in ftr['School'].values or away_team not in ftr['School'].values:
        matchups = matchups.drop(index)

KeyError: '[4] not found in axis'

In [269]:
matchups

,Home,Away,Home_Advantage
0,Minnesota,Indiana,2.0
1,Arkansas,Louisville,3.0
2,Alabama,Clemson,3.0
5,Akron,Bucknell,2.0
6,Wisconsin,Northwestern,3.0
7,Texas,Virginia,3.0
8,Georgia Tech,Mississippi St.,2.5
9,Washington,UCLA,2.0


In [270]:
for home_team, away_team,home_adj in matchups.values:
    print(home_team,away_team)
    
    sim_spread,sim_total = simulate_game(home_team, away_team,1000, home_adjustment=home_adj)
    sim_spread_nobart,sim_total_nobart = simulate_game(home_team, away_team,1000,home_adjustment=home_adj,use_bart=False)
    #eff_spread,eff_total = get_efficiency_pred(home_team, away_team)
    #Update matchups for the game
    matchups.loc[matchups['Home'] == home_team, 'Sim Spread'] = np.mean(sim_spread)
    matchups.loc[matchups['Home'] == home_team, 'Sim Totals'] = np.mean(sim_total)
    matchups.loc[matchups['Home'] == home_team, 'Sim Spread No Bart'] = np.mean(sim_spread_nobart)
    matchups.loc[matchups['Home'] == home_team, 'Sim Totals No Bart'] = np.mean(sim_total_nobart)
    #matchups.loc[matchups['Home'] == home_team, 'Eff Spread'] = eff_spread
    #matchups.loc[matchups['Home'] == home_team, 'Eff Totals'] = eff_total
#print(matchups)

Minnesota Indiana
Minnesota Wins:  0.1
Indiana Wins:  0.9
Ties:  0.0
Median Spread:  15.516688807582835
Median Total:  124.17301607211726
Average Spread:  15.632624362189285
Average Total:  124.64672819048731
Money Line Home Team:  900
Minnesota Wins:  0.224
Indiana Wins:  0.776
Ties:  0.0
Median Spread:  9.723033012054138
Median Total:  130.18116263651265
Average Spread:  9.894377580526104
Average Total:  130.2266700606025
Money Line Home Team:  346
Arkansas Louisville
Arkansas Wins:  0.321
Louisville Wins:  0.679
Ties:  0.0
Median Spread:  5.778294140380851
Median Total:  141.4469085306879
Average Spread:  6.330508907214561
Average Total:  141.82123366718466
Money Line Home Team:  212
Arkansas Wins:  0.357
Louisville Wins:  0.643
Ties:  0.0
Median Spread:  5.4142655578390375
Median Total:  142.84957572457304
Average Spread:  4.984796030931977
Average Total:  142.99714980588544
Money Line Home Team:  180
Alabama Clemson
Alabama Wins:  0.404
Clemson Wins:  0.596
Ties:  0.0
Median Sprea

In [ ]:
#matchups = matchups.iloc[:,0:6]

In [271]:
final_odds = pd.read_csv('12-03-2025 Odds.csv')

spreads = final_odds[(final_odds['Market'] == 'spreads') & (final_odds['Bookmaker'] == 'fanduel')]
#totals = final_odds[(final_odds['Market'] == 'totals') & (final_odds['Bookmaker'] == 'fanduel')]

In [272]:
#Join matchups with final odds based on team names
matchups = matchups.merge(spreads[['Team Modified','Point','Vegas Line']], left_on='Home', right_on='Team Modified', how='left')
matchups.rename(columns={'Point':'Home Spread','Vegas Line':'Home Vegas Line'}, inplace=True)
matchups.drop(columns=['Team Modified'], inplace=True)
matchups = matchups.merge(spreads[['Team Modified','Point','Vegas Line']], left_on='Away', right_on='Team Modified', how='left')
matchups.rename(columns={'Point':'Away Spread','Vegas Line':'Away Vegas Line'}, inplace=True)
matchups.drop(columns=['Team Modified'], inplace=True)

In [273]:
matchups['Proj Spread Home'] = ((matchups['Sim Spread'] + matchups['Sim Spread No Bart'])/2) * .25 + (matchups['Home Spread'] * .75)
matchups['Proj Spread Away'] = ((-matchups['Sim Spread'] + -matchups['Sim Spread No Bart'])/2) * .25 + (matchups['Away Spread'] * .75)

In [274]:
matchups['WP Home'] = matchups.apply(lambda x: calculate_win_probability(x['Sim Spread'], x['Home Spread']), axis=1)
matchups['WP Away'] = matchups.apply(lambda x: calculate_win_probability(-x['Sim Spread'], x['Away Spread']), axis=1)
matchups['EV Home'] = matchups.apply(lambda x: calculate_ev(x['WP Home'],x['Home Vegas Line']), axis=1)
matchups['EV Away'] = matchups.apply(lambda x: calculate_ev(x['WP Away'],x['Away Vegas Line']), axis=1)
matchups

,Home,Away,Home_Advantage,Sim Spread,Sim Totals,Sim Spread No Bart,Sim Totals No Bart,Home Spread,Home Vegas Line,Away Spread,Away Vegas Line,Proj Spread Home,Proj Spread Away,WP Home,WP Away,EV Home,EV Away
0,Minnesota,Indiana,2.0,15.632624,124.646728,9.894378,130.226670,8.5,-104.166667,-8.5,-117.647059,9.565875,-9.565875,0.056480,0.943520,-98.365114,73.986395
1,Arkansas,Louisville,3.0,6.330509,141.821234,4.984796,142.997150,2.5,-109.890110,-2.5,-109.890110,3.289413,-3.289413,0.197322,0.802678,-70.338195,51.338195
2,Alabama,Clemson,3.0,3.324912,149.575086,4.452331,147.702298,-10.5,-102.040816,10.5,-120.481928,-6.902845,6.902845,0.001062,0.998938,-109.779015,82.794952
3,Akron,Bucknell,2.0,-38.489467,165.481021,-19.729433,146.741709,-23.5,-120.481928,23.5,-102.040816,-24.902363,24.902363,0.999567,0.000433,82.916492,-109.910002
4,Wisconsin,Northwestern,3.0,-2.659644,145.573646,-0.547218,144.033914,-9.5,-109.890110,9.5,-109.890110,-7.525858,7.525858,0.064246,0.935754,-97.086643,78.086643
5,Texas,Virginia,3.0,-1.312162,138.683213,-1.853073,139.928503,-2.5,-112.359551,2.5,-107.526882,-2.270654,2.270654,0.395904,0.604096,-31.215137,12.631521
6,Georgia Tech,Mississippi St.,2.5,-3.469959,129.153048,-6.014689,131.118307,2.5,-102.040816,-2.5,-120.481928,0.689419,-0.689419,0.907689,0.092311,78.799310,-92.183976
7,Washington,UCLA,2.0,4.266259,126.901067,1.788525,129.301245,2.5,-120.481928,-2.5,-102.040816,2.631848,-2.631848,0.347344,0.652656,-42.962671,25.752515


12-2: 4/8

12-3: 
would play ucla, goergia tech, northwestern, clemson, louisvile, indiana (1/6)

12-4:


In [21]:
calculate_ev(0.572693,-120)

0.7206466666666671

In [645]:
matchups[matchups['EV Home'] > 0][['Home', 'Away','Proj Spread Home','Home Spread','Home Vegas Line','EV Home']].sort_values('EV Home',ascending=False)


,Home,Away,Proj Spread Home,Home Spread,Home Vegas Line,EV Home


In [646]:
matchups[matchups['EV Away'] > 0][['Home', 'Away','Proj Spread Away','Away Spread','Away Vegas Line','EV Away']].sort_values('EV Away',ascending=False)

,Home,Away,Proj Spread Away,Away Spread,Away Vegas Line,EV Away
3,Alabama,Saint Mary's,4.538509,5.5,-109.890110,43.709409
4,Maryland,Colorado St.,6.738853,7.5,-106.382979,23.165047
6,Michigan St.,New Mexico,6.917117,7.5,-109.890110,14.854529


In [102]:
matchups.to_clipboard(index = False)

In [12]:
proj_spread = 2.881274511 * .25 + 6.5 * .75
vegas_spread = 6.5
wp = calculate_win_probability(proj_spread, vegas_spread) 
print(wp)
calculate_ev(wp, -110)

0.5796664706967201


11.729958846311213

In [4]:
def calculate_units_to_bet(ev, bankroll):
    """
    Calculate the number of units to bet based on expected value (EV) per $100 bet.
    
    :param ev: Expected value in dollars per $100 bet (e.g., $10 means +10% EV).
    :param bankroll: Total available bankroll in dollars.
    :return: Number of units to bet.
    """
    # Calculate units to bet using the formula
    units_to_bet = (ev * bankroll) / 100
    
    # Ensure the bet size is not negative
    units_to_bet = max(0, units_to_bet)
    
    return units_to_bet

# Example usage
ev = 3.57  # Expected value of $10 per $100 bet
bankroll = 500  # Total bankroll in dollars

units = calculate_units_to_bet(ev, bankroll)
print(f"Recommended bet size: {units:.2f} units")


Recommended bet size: 17.85 units


In [451]:
import random

print(random.choice([0,1]))

0
